In [34]:
import numpy as np
import pandas as pd
import tqdm
import warnings
warnings.filterwarnings("ignore")

In [35]:
class NotFittedError(BaseException):
    def __init__(self, message):
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return self.message

class Perceptron:
    def __init__(self, regularization_strength=0.01):
        self.bias = 0
        self.weights = None
        self.learning_rate = 0.1
        self.regularization_strength = regularization_strength

    def fit(self, x:pd.DataFrame, y:pd.Series, epochs=10, verbose=False):
        self.weights = np.zeros(x.shape[1])
        disable = True if not verbose else False

        for _ in tqdm.tqdm(range(epochs), disable=disable):
            for j in range(x.shape[0]):
                y_predicted = self.predict(x.iloc[j])
                weight_gradient = (y[j] - y_predicted) * x.iloc[j]
                self.weights = self.weights - self.learning_rate * weight_gradient - self.learning_rate * self.regularization_strength * self.weights
                self.bias = self.bias + self.learning_rate * (y[j] - y_predicted)
                
    
    def predict(self, x:pd.Series):
        if self.weights is None:
            raise NotFittedError("Perceptron has not been trained")
        if x.shape != self.weights.shape:
            raise ValueError(f"Expected input of shape {self.weights.shape}, got {x.shape}")
        return self.step_activation(np.sum(x * self.weights) + self.bias)
    

    def step_activation(self, value):
        return 1 if value > 0 else 0


In [36]:
def truth_table(n:int, gate: str="or", num: int | None = None) -> pd.DataFrame:
    if gate not in ["or", "and"]:
        raise ValueError("Gate must be one of 'or', or 'and'")
    if n <= 1:
        raise ValueError("n must be greater than 1")
    table = {}
    for i in range(n):
        number = round((2 ** n // (2**i))/ 2)
        table[f"x{i}"] = ([0] * number + [1] * number) * (2**i)
    x = pd.DataFrame(table)
    if not num:
        y = x.any(axis=1,) if gate == "or" else x.all(axis=1)
    else:
        y = pd.Series([1 if i.count(1) >= num else 0 for i in x.itertuples()])        
    y.name = "y"
    y = y.astype(np.int16)
    return pd.concat([x, y], axis=1)

In [37]:
table = truth_table(5, num=4)

In [38]:
x = table.drop(columns="y")
y = table.y

In [39]:
def train_test_split(x:pd.DataFrame, y:pd.Series, test_split:float=0.2):
    train_size = 1 - test_split
    n = x.shape[0]
    train_size = int(n * train_size)
    x_train = x.iloc[:train_size]
    x_test = x.iloc[train_size:]
    y_train = x.iloc[:train_size]
    y_test = x.iloc[train_size:]
    return x_train, x_test, y_train, y_test
    

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
model = Perceptron(0.001)
model.fit(x, y, verbose=True)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 57.57it/s]
